In [1]:
import pandas as pd
df = pd.read_csv("../dataset/preprocessed/full.csv")


In [2]:
# Si prendono le righe che hanno reference non nulla e il campo 'city' non nullo. Si eliminano i 
# le righe che hanno la stessa tripletta ['session_id','reference','city']
mask = df['reference'].notnull() &(df['reference'].dtype != 'str') & (df['city'].notnull())
df_locations = df[mask]
df_locations = df_locations[['session_id','reference','city']].drop_duplicates()
df_locations = df_locations[df_locations['reference'].str.isnumeric() == True]

In [3]:
df_city = df_locations.groupby('city')['reference'].apply(lambda x: list(x))
df_city

city
's-Heerenberg, Netherlands                              [2010379, 53248, 53248, 2725638]
's-Hertogenbosch, Netherlands          [444516, 1387386, 4828612, 775223, 444516, 444...
100 Mile House, Canada                 [8163644, 8163644, 43791, 116043, 116043, 9954...
100 Mile Ranch, Canada                                                         [5210050]
10th of Ramadan City, Egypt                                                    [2431048]
25 de mayo, Argentina                               [8871972, 8871972, 8871972, 8871972]
6th of October City, Egypt             [4751702, 3477558, 92439, 1586313, 6267874, 55...
A Lama, Spain                                                                  [1786927]
A Pontenova, Spain                                             [185806, 949561, 6988358]
A Teixeira, Spain                                                              [1869767]
Aabenraa, Denmark                      [4044844, 1286666, 4044844, 54002, 4065276, 31...
Aachen, Germany 

In [4]:
from collections import Counter

In [5]:
# Si ordinano le accomodation in base alla frequenza in una città
df_city_occurrencies = pd.concat([pd.DataFrame([[index, dict(Counter(value).most_common())]], columns=['city', 'occurrencies']) for index, value in df_city.iteritems()], ignore_index=True)
#df_city_occurrencies

,city,occurrencies
0,"'s-Heerenberg, Netherlands","{'53248': 2, '2010379': 1, '2725638': 1}"
1,"'s-Hertogenbosch, Netherlands","{'163255': 22, '775223': 21, '444516': 18, '10..."
2,"100 Mile House, Canada","{'116043': 3, '8163644': 2, '44223': 2, '43791..."
3,"100 Mile Ranch, Canada",{'5210050': 1}
4,"10th of Ramadan City, Egypt",{'2431048': 1}
5,"25 de mayo, Argentina",{'8871972': 4}
6,"6th of October City, Egypt","{'55288': 5, '92439': 2, '1586313': 2, '626787..."
7,"A Lama, Spain",{'1786927': 1}
8,"A Pontenova, Spain","{'185806': 1, '949561': 1, '6988358': 1}"
9,"A Teixeira, Spain",{'1869767': 1}


In [6]:
df_test = pd.read_csv("/Users/claudiorussointroito/Documents/GitHub/recsysTrivago2019/dataset/original/test.csv")

In [7]:
import math

In [8]:
def get_submission_target(df):
    """Identify target rows with missing click outs."""

    mask = df["reference"].isnull() & (df["action_type"] == "clickout item")
    df_out = df[mask]

    return df_out

def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

In [64]:
# Submission considerando tutti i clickout del test e ordinandoli in base alla popolarità di 
# un'accomodation in quella città. Score: 0.324485
df_target = get_submission_target(df_test)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247|559056|1045096|1963879|693596|1967173|...,80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...
109,0339C84S24ET,89171d441a304,1541615683,36,clickout item,NaN,TR,"Antalya, Turkey",mobile,NaN,13361|5647680|116764|898719|8276346|9168|19325...,185|84|30|19|46|77|123|23|25|25|26|39|73|56|96...
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,clickout item,NaN,UK,"John o' Groats, United Kingdom",desktop,NaN,1193320|5488246|3858774|4552034|10620372|22696...,103|88|100|134|109|138|126|86
115,03LTH89QY623,7663406cf586c,1541554183,4,clickout item,NaN,CA,"Koloa, USA",desktop,NaN,241961|906477|991561|353701|1149665|77258|4943...,287|300|261|197|163|263|262|188|540|283|211|22...
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,clickout item,NaN,UK,"Warrington, United Kingdom",desktop,NaN,109938|164193|632366|1362450|1070666|164220|11...,45|67|78|60|58|57|86|68|57|55|148|92|66|55|61|...
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,clickout item,NaN,MX,"Puebla, Mexico",mobile,Sort By Popularity,42692|5116230|42876|4342578|42864|3148690|2123...,53|181|60|45|96|60|80|59|41|122|50|43|113|48|5...


In [11]:
df_target_copy = df_target.copy()
df_target_copy.loc[:,'impressions'] = df_target_copy['impressions'].apply(string_to_array)

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,"[2059240, 2033381, 1724779, 127131, 399441, 10...",70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,"[10884872, 7065316]",64|28
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,"[3565720, 2947584, 4115018, 2039671, 3836538, ...",16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,"[1451247, 559056, 1045096, 1963879, 693596, 19...",80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...
109,0339C84S24ET,89171d441a304,1541615683,36,clickout item,NaN,TR,"Antalya, Turkey",mobile,NaN,"[13361, 5647680, 116764, 898719, 8276346, 9168...",185|84|30|19|46|77|123|23|25|25|26|39|73|56|96...
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,clickout item,NaN,UK,"John o' Groats, United Kingdom",desktop,NaN,"[1193320, 5488246, 3858774, 4552034, 10620372,...",103|88|100|134|109|138|126|86
115,03LTH89QY623,7663406cf586c,1541554183,4,clickout item,NaN,CA,"Koloa, USA",desktop,NaN,"[241961, 906477, 991561, 353701, 1149665, 7725...",287|300|261|197|163|263|262|188|540|283|211|22...
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,clickout item,NaN,UK,"Warrington, United Kingdom",desktop,NaN,"[109938, 164193, 632366, 1362450, 1070666, 164...",45|67|78|60|58|57|86|68|57|55|148|92|66|55|61|...
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,clickout item,NaN,MX,"Puebla, Mexico",mobile,Sort By Popularity,"[42692, 5116230, 42876, 4342578, 42864, 314869...",53|181|60|45|96|60|80|59|41|122|50|43|113|48|5...


In [12]:
# Per ciascun clickout da predire, si ordinano le impressions in base alla popolarità per quella città

#from itertools import islice

for index,row in df_target_copy.iterrows():
    if(df_city_occurrencies[df_city_occurrencies['city'] == row['city']].shape[0] > 0):
        occ = df_city_occurrencies[df_city_occurrencies['city'] == row['city']]['occurrencies'].values[0]
        new_ord = {}
        impr = row['impressions']
        for i in impr:
            if i in occ:
                new_ord[i] = occ.get(i)
        new_ord = Counter(new_ord)
        #[key for key, _ in counter.most_common()] #convert the counter object to list with only references
        df_target_copy.at[index, 'impressions'] = [key for key, _ in new_ord.most_common()]


KeyboardInterrupt: 

In [ ]:
for index, row in df_target_copy.iterrows():
    df_target_copy.at[index, 'impressions'] =  " ".join(str(x) for x in row['impressions'])

In [38]:
df_t = df_target_copy[['user_id','session_id','timestamp','step','impressions']]
df_t

,user_id,session_id,timestamp,step,impressions
6,004A07DM0IDW,1d688ec168932,1541555799,7,"[399441, 127128, 2033381, 1167004, 1724779, 12..."
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,"[10884872, 7065316]"
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,"[4476010, 9343000, 7101352, 3843244, 5101686, ..."
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,"[2817590, 2012359, 3180004, 6434434, 4115018, ..."
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,"[1393443, 559091, 1177548, 559056, 1451247, 10..."
109,0339C84S24ET,89171d441a304,1541615683,36,"[5647680, 9500346, 8276346, 4500498, 5055970, ..."
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,"[1193320, 5488246, 3858774]"
115,03LTH89QY623,7663406cf586c,1541554183,4,"[1638525, 59566, 75269, 77258, 906477, 1017869..."
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,"[109938, 164193, 322491, 1070666, 492516, 1642..."
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,"[42692, 349091, 42876, 5702804, 6953534, 10129..."


In [ ]:
df_sub = df_t.copy()
df_sub.columns = ['user_id','session_id','timestamp','step','item_recommendations']

In [84]:
#df_sub.to_csv("../location_subm.csv", encoding='utf-8', index=False)

In [9]:
# Seconda submission considerando solo i clickout delle sessioni che non hanno riferimento numerico 
# nella sessione. Per il resto dei clickout è mantenuto l'ordine di apparizione nelle impressions
# Score: 0.41463  
def existsIndex(df, idx):
    try:
        df.loc[idx]
        return True
    except KeyError:
        return False
def func(x):
    y = x[x['reference'].isnull()]
    if len(y) == 1:
        return x
    
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

In [10]:
just_missing_refs = df_test[df_test['reference'].isnull()]
just_missing_refs = just_missing_refs[just_missing_refs['action_type'] == 'clickout item']

In [11]:
just_missing_refs

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,2059240|2033381|1724779|127131|399441|103357|1...,70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,10884872|7065316,64|28
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,2714480|4476010|3843244|3833012|9017890|198100...,74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,3565720|2947584|4115018|2039671|3836538|801409...,16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,1451247|559056|1045096|1963879|693596|1967173|...,80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...
109,0339C84S24ET,89171d441a304,1541615683,36,clickout item,NaN,TR,"Antalya, Turkey",mobile,NaN,13361|5647680|116764|898719|8276346|9168|19325...,185|84|30|19|46|77|123|23|25|25|26|39|73|56|96...
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,clickout item,NaN,UK,"John o' Groats, United Kingdom",desktop,NaN,1193320|5488246|3858774|4552034|10620372|22696...,103|88|100|134|109|138|126|86
115,03LTH89QY623,7663406cf586c,1541554183,4,clickout item,NaN,CA,"Koloa, USA",desktop,NaN,241961|906477|991561|353701|1149665|77258|4943...,287|300|261|197|163|263|262|188|540|283|211|22...
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,clickout item,NaN,UK,"Warrington, United Kingdom",desktop,NaN,109938|164193|632366|1362450|1070666|164220|11...,45|67|78|60|58|57|86|68|57|55|148|92|66|55|61|...
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,clickout item,NaN,MX,"Puebla, Mexico",mobile,Sort By Popularity,42692|5116230|42876|4342578|42864|3148690|2123...,53|181|60|45|96|60|80|59|41|122|50|43|113|48|5...


In [16]:
count_last_ref_numeric = []
count_last_ref_not_numeric_numeric_before = []
count_no_ref_numeric_in_session = []
for idx, row in just_missing_refs.iterrows():
    sess = row['session_id']
    i = 1
    while True:
        if not existsIndex(df, idx - i):
            count_no_ref_numeric_in_session.append(sess)
            break
        prev_row = df.loc[idx - i]
        if prev_row['session_id'] != sess:
            count_no_ref_numeric_in_session.append(sess)
            break
        if RepresentsInt(prev_row['reference']):
            if i == 1:
                count_last_ref_numeric.append(sess)
                break
            else:
                count_last_ref_not_numeric_numeric_before.append(sess)
                break
        else:
            i += 1

In [17]:
len(count_no_ref_numeric_in_session)

88961

In [43]:
df_t = df_test.copy()

In [44]:
df_no_ref = df_t[df_t['session_id'].isin(count_no_ref_numeric_in_session)]
mask = df_no_ref["reference"].isnull() & (df_no_ref["action_type"] == "clickout item")
df_no_ref = df_no_ref[mask]
df_no_ref.loc[:,'impressions'] = df_no_ref['impressions'].apply(string_to_array)

In [21]:
from tqdm import tqdm_notebook as tqdm

In [23]:
for index,row in tqdm(df_no_ref.iterrows()):
    if(df_city_occurrencies[df_city_occurrencies['city'] == row['city']].shape[0] > 0):
        occ = df_city_occurrencies[df_city_occurrencies['city'] == row['city']]['occurrencies'].values[0]
        new_ord = {}
        impr = row['impressions']
        for i in impr:
            if i in occ:
                new_ord[i] = occ.get(i)
        new_ord = Counter(new_ord)
        df_no_ref.at[index, 'impressions'] = [key for key, _ in new_ord.most_common()]
    
df_no_ref

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
319,05BAHQHA5RCI,9f279162e8d5c,1541633993,5,clickout item,NaN,DE,"Ulm, Germany",tablet,Best Value,"[2616752, 8954902, 2071844, 726906, 25860, 140...",59|86|70|58|86|58|58|57|78|68|55|90|79|64|112|...
513,0BZN2KFMHLZF,d9497eeb946dc,1541668916,1,clickout item,NaN,BE,"Hasselt, Belgium",mobile,NaN,"[39758, 133101, 39730, 107902, 3887112, 702923...",90|88|80|80|90|62|65|90|89|75|70|109|165|104|6...
554,0D1XFBRWUX7Z,35e873299a45f,1541586069,1,clickout item,NaN,AU,"Sydney, Australia",mobile,NaN,"[54803, 54953, 14211, 54982, 55093, 54794, 955...",163|126|158|177|145|85|107|158|132|189|187|131...
598,0EPVF4QE049Q,11b583578f7f3,1541611766,1,clickout item,NaN,FI,"Seinäjoki, Finland",mobile,NaN,"[368306, 369426, 4916202, 2045589, 1328820, 36...",123|152|59|148|105|147|98|70|95|80|140
599,0GY71I4BUBQW,334177bbc6525,1541638087,1,clickout item,NaN,AU,"Rome, Italy",desktop,NaN,"[17056, 17108, 848536, 3194342, 2200778, 17608...",81|95|126|79|186|112|53|112|56|214|142|77|71|1...
600,0H2RFN3FXM5A,e9fa318e4aad0,1541576918,1,clickout item,NaN,IN,"Bengaluru, India",mobile,NaN,"[5684276, 4775794, 1400686, 7204042, 3969446, ...",37|16|19|50|148|16|157|14|21|19|14|16|16|22|13...
602,0HG5FZNPFDYC,db55ce2d55626,1541571390,2,clickout item,NaN,RU,"Rostov-on-Don, Russia",mobile,NaN,"[1165724, 4726076, 1048438, 1370240, 3193776, ...",54|205|40|52|187|34|39|29|70|46|39|69|36|55|55...
604,0HPR8MG837GZ,9e3e3ac5954f3,1541648887,2,clickout item,NaN,BR,"Boston, USA",tablet,NaN,"[8504, 8500, 3248570, 79307, 67917, 73974, 849...",100|72|117|63|125|106|210|77|76|99|99|113|113|...
682,0L2CALL26C66,59bfbcb10146d,1541593658,1,clickout item,NaN,PL,"Alicante, Spain",desktop,NaN,"[40109, 6724, 6721, 147227, 32937, 80983, 1194...",30|36|42|88|46|85|49|57|45|49|56|53|37|63|127|...
801,0ME9OE1GS1Q9,7afb20460a368,1541591150,1,clickout item,NaN,BG,"Prague, Czech Republic",mobile,NaN,"[14243, 13699, 100667, 1487955, 80400, 80376, ...",61|45|62|32|42|57|54|135|58|79|85|49|69|129|10...


In [28]:
df_test_for_sub = get_submission_target(df_test)
print(df_test_for_sub.shape)

(253573, 12)


In [30]:
df_test_for_s = df_test_for_sub[~df_test_for_sub['session_id'].isin(count_no_ref_numeric_in_session)]
print(df_test_for_s.shape)

(164612, 12)


In [37]:
df_test_for_s.shape

(164612, 12)

In [ ]:
df_test_for_s.loc[:,'impressions'] = df_test_for_s['impressions'].apply(string_to_array)

In [38]:
df_submission = df_test_for_s.append(df_no_ref)
print(df_submission.shape)

(253573, 12)


In [39]:
df_submission

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
6,004A07DM0IDW,1d688ec168932,1541555799,7,clickout item,NaN,CO,"Santa Marta, Colombia",mobile,NaN,"[2059240, 2033381, 1724779, 127131, 399441, 10...",70|46|48|76|65|65|106|66|87|43|52|44|60|61|50|...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,clickout item,NaN,IN,"Nathdwara, India",mobile,NaN,"[10884872, 7065316]",64|28
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,clickout item,NaN,PH,"Iloilo City, Philippines",mobile,NaN,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",74|14|22|38|55|44|28|34|23|27|12|108|19|21|36|...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,clickout item,NaN,PL,"Wisla, Poland",mobile,Sort by Price,"[3565720, 2947584, 4115018, 2039671, 3836538, ...",16|18|20|21|22|22|28|28|28|30|30|33|33|35|35|3...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,clickout item,NaN,JP,"Yokohama, Japan",desktop,Hotel|Resort|Sort by Price,"[1451247, 559056, 1045096, 1963879, 693596, 19...",80|81|81|82|82|82|83|83|83|85|85|88|88|91|92|9...
109,0339C84S24ET,89171d441a304,1541615683,36,clickout item,NaN,TR,"Antalya, Turkey",mobile,NaN,"[13361, 5647680, 116764, 898719, 8276346, 9168...",185|84|30|19|46|77|123|23|25|25|26|39|73|56|96...
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,clickout item,NaN,UK,"John o' Groats, United Kingdom",desktop,NaN,"[1193320, 5488246, 3858774, 4552034, 10620372,...",103|88|100|134|109|138|126|86
115,03LTH89QY623,7663406cf586c,1541554183,4,clickout item,NaN,CA,"Koloa, USA",desktop,NaN,"[241961, 906477, 991561, 353701, 1149665, 7725...",287|300|261|197|163|263|262|188|540|283|211|22...
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,clickout item,NaN,UK,"Warrington, United Kingdom",desktop,NaN,"[109938, 164193, 632366, 1362450, 1070666, 164...",45|67|78|60|58|57|86|68|57|55|148|92|66|55|61|...
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,clickout item,NaN,MX,"Puebla, Mexico",mobile,Sort By Popularity,"[42692, 5116230, 42876, 4342578, 42864, 314869...",53|181|60|45|96|60|80|59|41|122|50|43|113|48|5...


In [41]:
for index, row in df_submission.iterrows():
    df_submission.at[index, 'impressions'] =  " ".join(str(x) for x in row['impressions'])
    
df_submission = df_submission[['user_id','session_id','timestamp','step','impressions']]
df_submission.columns = ['user_id','session_id','timestamp','step','item_recommendations']
#df_submission

,user_id,session_id,timestamp,step,item_recommendations
6,004A07DM0IDW,1d688ec168932,1541555799,7,2059240 2033381 1724779 127131 399441 103357 1...
8,009RGHI3G9A3,f05ab0de907e2,1541570940,2,10884872 7065316
10,00Y1Z24X8084,26b6d294d66e7,1541651823,2,2714480 4476010 3843244 3833012 9017890 198100...
15,01V3WDTDM5CU,07628a0f5be0b,1541575643,5,3565720 2947584 4115018 2039671 3836538 801409...
61,02AOAVF9PVYH,4a01c3afbc224,1541681278,46,1451247 559056 1045096 1963879 693596 1967173 ...
109,0339C84S24ET,89171d441a304,1541615683,36,13361 5647680 116764 898719 8276346 9168 19325...
111,0386OH8JDE1Q,e09591d07cdef,1541620536,2,1193320 5488246 3858774 4552034 10620372 22696...
115,03LTH89QY623,7663406cf586c,1541554183,4,241961 906477 991561 353701 1149665 77258 4943...
138,03VT0ODUTZB0,725e8adf70e86,1541632490,23,109938 164193 632366 1362450 1070666 164220 11...
314,03XH0JWCWHAM,73f4c417ff730,1541566143,176,42692 5116230 42876 4342578 42864 3148690 2123...


In [42]:
#df_submission.to_csv("../location_subm.csv", encoding='utf-8', index=False)